In [1]:
from data_loader import DataLoader
from data_cleaner import DataCleaner
from feature_extractor import FeatureExtractor
from model_trainer import ModelTrainer
from inferencer import Inferencer

from datetime import datetime
import joblib
from dotenv import load_dotenv
import os
import pandas as pd
load_dotenv(dotenv_path='nb-prod/.env')

True

In [2]:
data_loader = DataLoader(enstoe_api_key=os.getenv('ENTSOE_API_KEY'))
data_loader.update_df(out_df_filepath="data/bronze/df.parquet")

No data available between 2024-09-29 11:01:00+02:00 -> 2024-09-29 12:46:28.902130+02:00


In [3]:
DataCleaner.clean(
    in_df_filepath='data/bronze/df.parquet',
    out_df_filepath='data/silver/df.parquet',
)

In [4]:
FeatureExtractor.extract_features(
    in_df_filepath='data/silver/df.parquet',
    out_df_filepath='data/gold/df.parquet',
)

In [7]:
reg = ModelTrainer.fit(n_estimators=100)
results_df, mape = ModelTrainer.backtest(
    Xy_filepath='data/gold/df.parquet',
    model=reg,
    starting_ts=pd.Timestamp(datetime.now() - pd.Timedelta(30, 'd'), tz='Europe/Zurich'),
    use_every_nth_ts=100,
)
joblib.dump(reg, 'model.joblib')
print(f'Backtested MAPE: {mape:.2f}%')

100%|█████████████████████████████████████████████| 7/7 [00:02<00:00,  3.39it/s]

Backtested MAPE: 5.35%


In [8]:
inferencer = Inferencer(model_filepath='model.joblib')
yhat = inferencer.predict(in_df_filepath='data/gold/df.parquet')
yhat.head(3)

,predicted_24h_later_load
2024-09-27 11:00:00+02:00,6778.003858
2024-09-27 12:00:00+02:00,6936.409836
2024-09-27 13:00:00+02:00,6945.083698
